In [1]:
# import torch
# torch.cuda.is_available()
# torch.backends.cudnn.enabled

In [2]:
from chamferdist import ChamferDistance

In [3]:
from src.model_point_torch import *

Eval not available.


In [4]:
opt = parse_config()
opt = pre_process(opt)

In [5]:
opt.batch_size = 16

In [6]:
opt

Namespace(activate_eval=0, activation='ReLU', argment_mode=0, argment_noise=0.01, batch_norm='ln', batch_size=16, beta1_des=0.9, category='chair', checkpoint_path=None, cuda='-1', data_path='data', data_size=10000, debug=99, device='cuda:-1', do_evaluation=1, drop_last=False, eval_step=50, fp16='None', gradient_accumulation_steps=1, hidden_size=[[64, 64, 128, 256, 1024], [512, 256, 128, 64]], langevin_clip=1, langevin_decay=0, learning_mode=0, lr=0.0005, lr_decay=0.998, mode='train', net_type='default', noise_decay=0, normalize='ebp', num_chain=1, num_point=2048, num_steps=2000, output_dir='output\\pytorch\\chair_default_medium_Oct21_17-32-29', point_dim=3, random_sample=1, ref_sigma=0.3, sample_step=64, seed=666, shuffle=True, stable_check=1, step_size=0.01, swap_axis=True, test_size=16, visualize_mode=0, warm_start=0)

In [7]:
train_data = util_torch.PointCloudDataSet(opt)
data_collator = util_torch.PointCloudDataCollator(opt)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=opt.batch_size, drop_last=opt.drop_last, 
    shuffle=opt.shuffle, collate_fn = data_collator, num_workers=torch.cuda.device_count() * 4)

In [8]:
from new.models import *

In [9]:
net_e = NetE()

In [10]:
net_g = NetG()

In [11]:
a = torch.randn(16,1024)

In [12]:
# batch = next(iter(train_loader))

In [13]:
batch = torch.randn(16, 3, 2048)

In [14]:
net_e(a).shape

torch.Size([16, 1])

In [15]:
net_g(a).shape

torch.Size([16, 3, 2048])

In [16]:
net_wrapper = NetWrapper()

In [17]:
a.shape

torch.Size([16, 1024])

In [18]:
z_plus, z_grad_norm = net_wrapper.sample_langevin_prior_z(a, net_wrapper.netE)

In [19]:
z_plus.shape

torch.Size([16, 1024])

In [20]:
net_wrapper(a).shape

torch.Size([16, 1024])

In [21]:
batch.shape

torch.Size([16, 3, 2048])

In [22]:
net_wrapper(a, x = batch, prior = False)

x_hat.shape torch.Size([16, 3, 2048]) torch.Size([16, 3, 2048])
x_hat.shape torch.Size([16, 3, 2048]) torch.Size([16, 3, 2048])
x_hat.shape torch.Size([16, 3, 2048]) torch.Size([16, 3, 2048])
x_hat.shape torch.Size([16, 3, 2048]) torch.Size([16, 3, 2048])
x_hat.shape torch.Size([16, 3, 2048]) torch.Size([16, 3, 2048])
x_hat.shape torch.Size([16, 3, 2048]) torch.Size([16, 3, 2048])
x_hat.shape torch.Size([16, 3, 2048]) torch.Size([16, 3, 2048])
x_hat.shape torch.Size([16, 3, 2048]) torch.Size([16, 3, 2048])
x_hat.shape torch.Size([16, 3, 2048]) torch.Size([16, 3, 2048])
x_hat.shape torch.Size([16, 3, 2048]) torch.Size([16, 3, 2048])
x_hat.shape torch.Size([16, 3, 2048]) torch.Size([16, 3, 2048])
x_hat.shape torch.Size([16, 3, 2048]) torch.Size([16, 3, 2048])
x_hat.shape torch.Size([16, 3, 2048]) torch.Size([16, 3, 2048])
x_hat.shape torch.Size([16, 3, 2048]) torch.Size([16, 3, 2048])
x_hat.shape torch.Size([16, 3, 2048]) torch.Size([16, 3, 2048])
x_hat.shape torch.Size([16, 3, 2048]) to

tensor([[ 0.1749,  0.8837,  0.4786,  ...,  0.3097,  0.6597, -0.8071],
        [ 0.0701, -0.4127, -0.6196,  ...,  0.4455,  1.0765,  2.1776],
        [ 0.7872,  2.5924,  0.3296,  ...,  0.0264, -0.5068, -0.9783],
        ...,
        [ 0.6432, -1.3634, -0.2784,  ..., -1.1733, -0.3474,  1.0219],
        [-1.5006, -0.6324, -0.1700,  ..., -1.8696, -0.7499,  0.1908],
        [ 1.0390,  0.8476, -0.7306,  ..., -0.9720, -1.3411, -0.6321]])

In [23]:
x_plus = net_g(a)

In [24]:
x_plus.shape

torch.Size([16, 3, 2048])

In [25]:
batch.transpose(1,2).shape

torch.Size([16, 2048, 3])

In [26]:
 from chamferdist import ChamferDistance

In [27]:
chamferDist = ChamferDistance()

In [28]:
loss = chamferDist(batch.transpose(1,2), x_plus.transpose(1,2))

In [29]:
loss.backward()

In [30]:
from new.params import *

In [31]:
x_hat = net_g(a)

In [32]:
x_hat.shape

torch.Size([16, 3, 2048])

In [33]:
batch.shape

torch.Size([16, 3, 2048])

In [34]:
g_log_lkhd = 1.0 / (2.0 * g_llhd_sigma * g_llhd_sigma) * net_wrapper.loss_fun(x_hat.transpose(1,2), batch.transpose(1,2))
